# Plot paired and unpaired synapses

This notebook assumes that all of the studies are orginized into a standard Python data structure. All of the plots are done using plot.ly, so you should look at the appropriate documentation for that if you want to change them.

The variable `study_list` constains a list of studies, where each study is represented as a pyton dictionary. The keys in the dictionary currently include the study name, the subject id, the type of study (aversion, control), and if the fish learned or not. 

The python command `list(study_list[0])` will return a list of the current values that are available for a study.

The study dictionary may also contain a set of calculated synapse pairings.  Each pair is represented as a python dictionary, whose key is the radius with which the pairing was determined, and whose value is a PANDA, with the synapse positions of the pair. Each PANDA has included in it a set of useful values from the study, including the study id, the type of study, the radius for the pair, etc.  Pairs are also transformed using the alignment points to put them into a common coordinate space.  The sets of pairs provided are:

* PairedBefore
* PairedAfter
* UnpairedBefore
* UnpairedAfter
* AlignedPairedBefore
* AlignedPairedAfter
* AlignedUnpairedBefore
* AlignedUnpairedAfter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load up all of the libraries we need

import synapse_plot_utils as sp
import synapse_plot_config as plot_config
from synapse_plot_config import plot_synapses, trace_color, position_layout, step_buttons

import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

<H2> Useful Global Variables

In [ ]:
study_list = ['SynStd6025','SynStd8013','SynStd8071','SynStd6791','SynStd7951','SynStd7949','SynStd6493','SynStd7943',
          'SynStd6479','SynStd6481','SynStd6485','SynStd6483', 'SynStd6477', 'SynStdR5Y',
          'SynStdJK6','SynStdJKP','SynStdJKE','SynStdJJY','SynStdJJP','SynStdJJE','SynStdJJ6',
          'SynStdRVA','SynStdRVE','SynStdRVJ','SynStdRVP','SynStdRVT']

syn_pair_radii = (4, 300.0,) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold


# Load Data

You have two choices, you can reload data from a previous notebook run, or regenerate all of the pairs with data from the catalog. If there have been new studies added, or any data has changed, you must regenerate all the pairs. In general, you should probably only use the save and 

<h3> Restore data from previous run

In [ ]:
dumpfile = 'pairs-dump.pkl'
paired_studies = sp.restore_studies(dumpfile)

<H3> Get Study Pairs from Catalog

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [ ]:
dumpfile = 'pairs-dump.pkl'

all_studies = sp.get_studies()

learners = sp.group_studies(all_studies, group='Type')['learner']
nonlearners = sp.group_studies(all_studies, group='Type')['nonlearner']
aligned = sp.group_studies(all_studies, group='Aligned')[True]

print('{0} learners'.format(len(learners)))
print('{0} nonlearners'.format(len(nonlearners)))
print('{0} aligned'.format(len(aligned)))

# Lets just get the learners and nonlearners for now.
paired_studies = [  s for s in all_studies if s['Study'] in study_list]

sp.compute_pairs(paired_studies, syn_pair_radii)

dumpfile = 'pairs-dump.pkl'

sp.dump_studies(paired_studies, dumpfile)


<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired Before','Paired After']
c1, c2, c3, c4, c5, c6, c7 = [],[],[],[],[],[],[]

syn_pair_radii = list(paired_studies[0]['UnpairedBefore'])

for s in paired_studies:
    for r in syn_pair_radii:
        c1.append(s['Study'])
        c2.append(s['Type'])
        c3.append(r)
        c4.append(len(s['UnpairedBefore'][r]['Data'].index))
        c5.append(len(s['UnpairedAfter'][r]['Data'].index))
        c6.append(len(s['PairedBefore'][r]['Data'].index))
        c7.append(len(s['PairedAfter'][r]['Data'].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
pair_distance = { 'learner': pd.Series(), 'nonlearner' : pd.Series(), 'control' : pd.Series()}

rmax = max(s['PairedAfter'])

for s in paired_studies:
    # Get the pairing for the maximum radius...
    before = s['PairedBefore'][rmax]['Data']
    after = s['PairedAfter'][rmax]['Data']
    t = s['Type']
    pair_distance[t] = pair_distance[t].append(pd.Series(np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)))

data=  [ 
    go.Histogram(x=pair_distance['learner'], name='Learners Radias {0}'.format(rmax)), 
    go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radias {0}'.format(rmax)),
    go.Histogram(x=pair_distance['control'], name='Control Radias {0}'.format(rmax))
]

hist_layout = go.Layout(
    title='Pair Distance Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<h1> Histogram of pair distances - past a certain pairing threshold 

In [ ]:
thres_pair=20.0 # Threshold for the minimum value to display on the histogram

hist_data_l_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))
hist_data_nl_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))

# Normalized
hist_data_l_N_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")
hist_data_nl_N_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")

hist_layout_Norm = go.Layout(
    title='Pair Distance Histogram Radius Normalized {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig_thres = go.Figure(data=[hist_data_l_thres, hist_data_nl_thres], layout=hist_layout)
hist_fig_Norm_thres = go.Figure(data=[hist_data_l_N_thres, hist_data_nl_N_thres], layout=hist_layout_Norm)

py.offline.iplot(hist_fig_thres, filename='pair-distance_thresholded')
py.offline.iplot(hist_fig_Norm_thres, filename='pair-distance_Norm_thresholded')

<h1> Histogram of intensity ratios

In [ ]:
r  = min(paired_studies[0]['UnpairedBefore'])

core_ratio = { 'learner': pd.Series(), 'nonlearner' : pd.Series(), 'control': pd.Series()}
core = { 'learner': pd.Series(), 'nonlearner' : pd.Series(), 'control': pd.Series()}

for s in paired_studies:
    before = s['PairedBefore'][r]['Data']
    after = s['PairedAfter'][r]['Data']
    t = s['Type']
    core_ratio[t] = core_ratio[t].append(pd.Series(before['core'] / after['core']))
    core[t] = core_ratio[t].append(pd.Series(before['core']))
    core[t] = core_ratio[t].append(pd.Series(after['core']))

data = [
    go.Histogram(x=core_ratio['learner'], name='Learners Radias {0}'.format(r)),
    go.Histogram(x=core_ratio['nonlearner'], name='Nonlearners Radias {0}'.format(r)),
    go.Histogram(x=core_ratio['control'], name='Control Radias {0}'.format(r))
]
    
hist_layout = go.Layout(
    title='Pair Ratio Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='core-intensity')

<H1> Plot sets of synapses...

In [ ]:
# Get all radii for a specific study


s_names = ['SynStd6025','SynStd8013','SynStd8071','SynStd6791','SynStd7951','SynStd7949','SynStd6493','SynStd7943',
          'SynStd6479','SynStd6481','SynStd6485','SynStd6483', 'SynStd6477', 'SynStdR5Y',
          'SynStdJK6','SynStdJKP','SynStdJKE','SynStdJJY','SynStdJJP','SynStdJJE','SynStdJJ6',
          'SynStdRVA','SynStdRVE','SynStdRVJ','SynStdRVP','SynStdRVT']
# Removed SynStd6475 because the registration of the original image seems incorrect - somehow shifted??
# SynStdJK6 was bounced back on 1/18/18 so take it with a grain of salt for now

s_list = [s for s in paired_studies if s['Study'] in s_names]
# Pick which traces you would like displayed


tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter','AlignmentPts']
#tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter']
#tracelist = ['UnpairedBefore','UnpairedAfter']
print('Plotting {0} studies'.format(len(s_list)))
study_types = sp.group_studies(s_list, group='Type')

print('{0} Learners'.format(len(study_types['learner'])))
print('{0} NonLearners'.format(len(study_types['nonlearner'])))
print('{0} Controls'.format(len(study_types['control'])))


#Pick the slider mode...
plotmode = 'Study'
plotmode = 'Trace'
#plotmode = 'Studyset'


fig = plot_synapses(s_list, tracelist, plotmode, centroid=False) 

plot_file_name = 'synapse-plot'
py.offline.iplot(fig, filename=plot_file_name)
    
#py.offline.plot(fig,  filename =plot_file_name, output_type='file')
    

<h2> Plot summary of centroids

In [ ]:
centroidlist = []
studylist = s_list
tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter','AlignmentPts']

# Use the smallest available radius as the default if one is not provided.
radius =  min(studylist[0][tracelist[0]])

for t in tracelist: 
    if t != 'AlignmentPts':
        centroidlist.append(t + 'Centroid')
before_traces, after_traces = [], []

cnt_before = cnt_after = 0
before_x = before_y = before_z = 0
after_x = after_y = after_z = 0
maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
    
for t in centroidlist:
    x,y,z, text = [],[],[],[]
    for s in studylist:
        x.append(s[t][radius]['Data']['x'][0])
        y.append(s[t][radius]['Data']['y'][0])
        z.append(s[t][radius]['Data']['z'][0])
        text.append(s['Study'])
        
    maxx = max(x + [maxx])
    maxy = max(y + [maxy])
    maxz = max(z + [maxz])
    minx = min(x + [minx])
    miny = min(y + [miny])
    minz = min(z + [minz])
    
    centroid_trace = go.Scatter3d(
        x = x, y = y, z = z,
        mode='markers',
        name = '{0}'.format(t),
        text = text,
        marker=dict(
            size = 6, color = trace_color(t),
            line=dict(color='black', width=0.8))
        )

    if 'Before' in t:
        before_x = sum(x) + before_x
        before_y = sum(y) + before_y
        before_z = sum(z) + before_z
        cnt_before = cnt_before + len(x)
        before_traces.append(centroid_trace)
    else:
        after_x = sum(x) + after_x
        after_y = sum(y) + after_y
        after_z = sum(z) + after_z
        cnt_after = cnt_after + len(x)
        after_traces.append(centroid_trace)
        
summary_trace = go.Scatter3d(
    x = [ before_x / cnt_before ], y = [ before_y / cnt_before ], z = [ before_z / cnt_before ],
    mode = 'markers',
    name = 'Centroids Before',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['orange']
     )
)
before_traces.append(summary_trace)

summary_trace = go.Scatter3d(
    x = [ after_x / cnt_after ],  y = [ after_y / cnt_after ], z = [ after_z / cnt_after ],
    mode = 'markers',
    name = 'Centroids After',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['blue']
     )
)
after_traces.append(summary_trace)

masks = {'study' : 
         { 'centroid' : 
          {
    'all' : { 'study' : [True] * (len(before_traces) + len(after_traces)) },
    'before' : { 'study' : [True] * len(before_traces) + [False] * len(after_traces)},
    'after' : { 'study' : [False] * len(before_traces) + [True] * len(after_traces)}
          }
         }
    }

layout = position_layout(minx, miny, minz, maxx, maxy, maxz)
layout['title'] = 'Centroids'
layout['updatemenus'] = step_buttons('study', masks, 'centroid')

fig = dict(data=before_traces + after_traces, layout=layout)

plot_file_name = 'centroid-plot'

py.offline.iplot(fig, filename=plot_file_name)
    
#py.offline.plot(fig,  filename =plot_file_name, output_type='file')